In [1]:
include("AZP_env.jl")

score_test (generic function with 1 method)

In [2]:
function calc_Kt_sym(history::Vector{Int}, env::Env)
    MV = []
    his = copy(history)
    #println(length(his))
    for it in 1:length(his)
        sw = pop!(his)
        if(sw==1)
            push!(MV, env.H_0)
        elseif(sw==2)
            push!(MV, env.V_t*sx)
        elseif(sw==3)
            A = pop!(MV)
            B = pop!(MV)
            C = A + B
            push!(MV, C)
        elseif(sw==4)
            A = pop!(MV)
            B = pop!(MV)
            C = -1im*(A*B - B*A)
            push!(MV, C)
        elseif(sw==5)
            A = pop!(MV)
            B = pop!(MV)
            C = (A*B + B*A)/2
            push!(MV, C)
        elseif(sw==6)
            A = pop!(MV)
            try
                #B = A.integrate((x, 0, x))/env.Ω
                B = A.integrate(x)/env.Ω
            catch
                B = A
            end
            push!(MV, B)
        end
        #@show MV
    end
    return MV[end]
end

calc_Kt_sym (generic function with 1 method)

In [6]:
en = init_Env_quick(["15", "64", "5", "0.25"], Any[0.3, Int(72), 1.0])

Env(15, 200, 2, 3, 1, 6, 90, 64, 7, 5, 1000, 200, 500, 1, 1.0f-5, 0.9f0, 512, 0.3f0, 0.25f0, 0.1f0, 1.0f0, 100, 4, 10.0f0, 0.4f0, 1.0f0, 0.7f0, 0.5f0, ComplexF32[-2.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im -0.7f0 + 0.0f0im; 0.0f0 - 0.0f0im 1.0f0 + 0.0f0im -0.7f0 + 0.0f0im 0.0f0 + 0.0f0im; 0.0f0 - 0.0f0im -0.7f0 - 0.0f0im 1.0f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.7f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 + 0.0f0im], ComplexF32[0.0f0 + 0.0f0im -0.4f0 + 0.0f0im -0.4f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.4f0 - 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im -0.4f0 + 0.0f0im; -0.4f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 + 0.0f0im -0.4f0 + 0.0f0im; 0.0f0 - 0.0f0im -0.4f0 - 0.0f0im -0.4f0 - 0.0f0im 0.0f0 + 0.0f0im], 0.0062831854f0, 72, 1.0f0, 1.0f-6)

In [9]:
t = collect(0:en.Ω*en.dt:2pi)

100-element Vector{Float64}:
 0.0
 0.06283185631036758
 0.12566371262073517
 0.18849556893110275
 0.25132742524147034
 0.3141592815518379
 0.3769911378622055
 0.4398229941725731
 0.5026548504829407
 0.5654867067933083
 ⋮
 5.71769892424345
 5.780530780553818
 5.843362636864185
 5.906194493174553
 5.9690263494849205
 6.031858205795288
 6.094690062105656
 6.157521918416023
 6.220353774726391

In [42]:
hist = [3, 1, 4, 2, 2]
@time Ks = calc_Kt_sym(hist, en)

  0.393549 seconds (460.89 k allocations: 31.670 MiB, 98.41% compilation time)


4×4 Matrix{Sym}:
  -2.00000000000000                   0  …  -0.699999988079071
                  0    1.00000000000000                      0
                  0  -0.699999988079071                      0
 -0.699999988079071                   0                      0

In [43]:
@time K0 = convert(Matrix{ComplexF32}, Ks)

  0.000431 seconds (289 allocations: 7.438 KiB)


4×4 Matrix{ComplexF32}:
 -2.0+0.0im   0.0+0.0im   0.0+0.0im  -0.7+0.0im
  0.0+0.0im   1.0+0.0im  -0.7+0.0im   0.0+0.0im
  0.0+0.0im  -0.7+0.0im   1.0+0.0im   0.0+0.0im
 -0.7+0.0im   0.0+0.0im   0.0+0.0im   0.0+0.0im

In [32]:
#K0 = N(Ks.subs(x,t[1]))

4×4 Matrix{Real}:
 0     0.04  0.04  0
 0.04  0     0     0.04
 0.04  0     0     0.04
 0     0.04  0.04  0

In [37]:
@time Kt = [Hermitian(ComplexF32.(N(Ks.subs(x,t[i]))-K0)) for i in 1:en.t_step]
@time Kt = [Hermitian(convert(Matrix{ComplexF32}, Ks.subs(x,t[i]))-K0) for i in 1:en.t_step]
println("")

  3.611101 seconds (691.99 k allocations: 22.648 MiB, 0.92% compilation time)


  1.071003 seconds (72.37 k allocations: 3.039 MiB, 2.69% compilation time)

